---

**Analytical Validation Tests for 2D FEM Solver**

- Poiseuille Flow: in-plane shear, no-slip at walls
- Couette Flow: in-plane shear, wall velocity
- Bernoulli Venturi: inertia terms
---

Pressure-driven flow in $x$-direction with no-slip walls as $y$-boundary condition.

**Setup:**
- Body force $f_x$ drives flow in $x$-direction (equivalent to pressure gradient)
- Periodic boundary conditions in $x$ (that is why we cannot directly use pressure gradient to impose flow)
- No-slip walls at $y=0$ and $y=L_y$: $j_x = j_y = 0$
- In-plane viscous diffusion terms active (`R23xy`, `R23yx`)
- Wall stress terms $\tau_{xz}, \tau_{yz}$ deactivated (`R24x`, `R24y`)

**Analytical solution:**

$$u(y) = \frac{\rho \cdot f_x}{2\mu} \cdot (y - y_{lo}) \cdot (y_{hi} - y)$$

where $y_{\mathrm{lo}} = -\Delta y/2$ and $y_{\mathrm{hi}} = L_y + \Delta y/2$ account for BCs applied at ghost cell centers.

$\rightarrow$ **[YAML File](../../tests/configs/poiseuille_2d_body_force.yaml)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from GaPFlow.problem import Problem

problem = Problem.from_yaml("../../tests/configs/poiseuille_2d_body_force.yaml")
problem.q[0][:] = 1.0  # Uniform density
problem.q[1][:] = 0.0  # Zero momentum
problem.q[2][:] = 0.0
problem.run()

Compare with analytical solution:

In [ ]:
# Extract velocity profile
rho = problem.q[0][1:-1, 1:-1]
jx = problem.q[1][1:-1, 1:-1]
vx = (jx / rho).mean(axis=0)  # Average over x

# Parameters
Ly, Ny = problem.grid['Ly'], problem.grid['Ny']
dy = Ly / Ny
mu = problem.prop['shear']
f_x = problem.prop['force_x']
rho_m = rho.mean()

# Analytical solution (BCs at ghost cell centers)
y = np.linspace(dy/2, Ly - dy/2, Ny)
y_lo, y_hi = -dy/2, Ly + dy/2
u_analytical = (rho_m * f_x) / (2 * mu) * (y - y_lo) * (y_hi - y)

# Error
l2_error = np.sqrt(((vx - u_analytical)**2).mean()) / u_analytical.max()
print(f"L2 relative error: {l2_error:.2e}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white', constrained_layout=True)

# 1. Velocity profile
axes[0].plot(u_analytical, y, 'b-', lw=2, label='Analytical')
axes[0].plot(vx, y, 'ro', ms=4, label='Simulated')
axes[0].set_xlabel('Velocity u [m/s]')
axes[0].set_ylabel('y [m]')
axes[0].set_title('Velocity Profile')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Error profile
rel_error = np.abs(vx - u_analytical) / u_analytical.max() * 100
axes[1].plot(rel_error, y, 'r-', lw=2)
axes[1].set_xlabel('Relative Error [%]')
axes[1].set_ylabel('y [m]')
axes[1].set_title('Error Profile')
axes[1].ticklabel_format(axis='x', style='scientific', scilimits=(-2, 2))
axes[1].grid(True, alpha=0.3)

# 3. 2D velocity field
Lx, Nx = problem.grid['Lx'], problem.grid['Nx']
vx_2d = (jx / rho)
x = np.linspace(dy/2, Lx - dy/2, Nx)
X, Y = np.meshgrid(x, y, indexing='ij')
im = axes[2].pcolormesh(X, Y, vx_2d, cmap='viridis', shading='auto')
axes[2].set_xlabel('x [m]')
axes[2].set_ylabel('y [m]')
axes[2].set_title('$v_x$ field')
plt.colorbar(im, ax=axes[2], label='[m/s]')

plt.suptitle('2D Poiseuille Flow Validation', fontweight='bold')
plt.show()

## Couette Flow

Shear-driven flow with stationary bottom wall and moving top wall.

**Setup:**
- Top wall moving with velocity $U$ (implemented as Dirichlet BC: $j_x = \rho \cdot U$)
- Bottom wall stationary: $j_x = j_y = 0$
- Periodic boundary conditions in $x$
- In-plane viscous diffusion terms active (`R23xy`, `R23yx`)
- No body force

**Analytical solution:**

$$u(y) = U \cdot \frac{y - y_{lo}}{y_{hi} - y_{lo}}$$

where $y_{\mathrm{lo}} = -\Delta y/2$ and $y_{\mathrm{hi}} = L_y + \Delta y/2$ account for BCs applied at ghost cell centers.

$\rightarrow$ **[YAML File](../../tests/configs/couette_2d_wall_velocity.yaml)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from GaPFlow.problem import Problem

problem_couette = Problem.from_yaml("../../tests/configs/couette_2d_wall_velocity.yaml")
problem_couette.q[0][:] = 1.0  # Uniform density
problem_couette.q[1][:] = 0.0  # Zero momentum
problem_couette.q[2][:] = 0.0
problem_couette.run()

Compare with analytical solution:

In [ ]:
# Extract velocity profile
rho_c = problem_couette.q[0][1:-1, 1:-1]
jx_c = problem_couette.q[1][1:-1, 1:-1]
vx_c = (jx_c / rho_c).mean(axis=0)  # Average over x

# Parameters
Ly_c, Ny_c = problem_couette.grid['Ly'], problem_couette.grid['Ny']
dy_c = Ly_c / Ny_c
rho_m_c = rho_c.mean()

# Wall velocity from BC (jx_wall / rho = U)
U_wall = problem_couette.grid['bc_yN_D_val'][1] / rho_m_c

# Analytical solution (linear profile, BCs at ghost cell centers)
y_c = np.linspace(dy_c/2, Ly_c - dy_c/2, Ny_c)
y_lo_c, y_hi_c = -dy_c/2, Ly_c + dy_c/2
u_analytical_c = U_wall * (y_c - y_lo_c) / (y_hi_c - y_lo_c)

# Error
l2_error_c = np.sqrt(((vx_c - u_analytical_c)**2).mean()) / u_analytical_c.max()
print(f"L2 relative error: {l2_error_c:.2e}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white', constrained_layout=True)

# 1. Velocity profile
axes[0].plot(u_analytical_c, y_c, 'b-', lw=2, label='Analytical')
axes[0].plot(vx_c, y_c, 'ro', ms=4, label='Simulated')
axes[0].set_xlabel('Velocity u [m/s]')
axes[0].set_ylabel('y [m]')
axes[0].set_title('Velocity Profile')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Error profile
rel_error_c = np.abs(vx_c - u_analytical_c) / u_analytical_c.max() * 100
axes[1].plot(rel_error_c, y_c, 'r-', lw=2)
axes[1].set_xlabel('Relative Error [%]')
axes[1].set_ylabel('y [m]')
axes[1].set_title('Error Profile')
axes[1].ticklabel_format(axis='x', style='scientific', scilimits=(-2, 2))
axes[1].grid(True, alpha=0.3)

# 3. 2D velocity field
Lx_c, Nx_c = problem_couette.grid['Lx'], problem_couette.grid['Nx']
vx_2d_c = (jx_c / rho_c)
x_c = np.linspace(dy_c/2, Lx_c - dy_c/2, Nx_c)
X_c, Y_c = np.meshgrid(x_c, y_c, indexing='ij')
im = axes[2].pcolormesh(X_c, Y_c, vx_2d_c, cmap='viridis', shading='auto')
axes[2].set_xlabel('x [m]')
axes[2].set_ylabel('y [m]')
axes[2].set_title('$v_x$ field')
plt.colorbar(im, ax=axes[2], label='[m/s]')

plt.suptitle('2D Couette Flow Validation', fontweight='bold')
plt.show()

## Bernoulli Venturi Nozzle

Inviscid flow through a symmetric venturi channel, validating convective momentum terms.

**Setup:**
- Symmetric venturi: $h_{\mathrm{inlet}} = 1\,\mathrm{mm} \to h_{\mathrm{throat}} = 0.5\,\mathrm{mm} \to h_{\mathrm{outlet}} = 1\,\mathrm{mm}$
- Smooth cosine transitions between sections
- DH EOS (nearly incompressible)
- Inlet: prescribed mass flux $j_x = \rho \cdot v = 5000\,\mathrm{kg/(m^2 \cdot s)}$
- Outlet: prescribed density $\rho = 1000\,\mathrm{kg/m^3}$
- Convective terms active (`R22xx`, `R22yx`, etc.)
- Custom topography via `topo.set_mapped_height()`

**Analytical predictions (Bernoulli equation):**

$$v \cdot h = \mathrm{const} \quad \Rightarrow \quad v_{\mathrm{throat}} = v_{\mathrm{inlet}} \cdot \frac{h_{\mathrm{inlet}}}{h_{\mathrm{throat}}} = 2 \cdot v_{\mathrm{inlet}}$$

$$p + \frac{1}{2}\rho v^2 = \mathrm{const} \quad \Rightarrow \quad \Delta p = \frac{1}{2}\rho(v_{\mathrm{throat}}^2 - v_{\mathrm{inlet}}^2)$$

$\rightarrow$ **[YAML File](../../tests/configs/bernoulli_venturi.yaml)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from GaPFlow.problem import Problem
from GaPFlow.models.pressure import eos_pressure

# Geometry parameters
H_INLET, H_THROAT = 1.0e-3, 0.5e-3
X_RAMP_START, X_RAMP_END = 0.03, 0.07
THROAT_HALF_WIDTH = 0.005
RHO0, V_INLET = 1000.0, 5.0

def venturi_topography(xx):
    """Symmetric venturi with cosine transitions."""
    x_mid = (X_RAMP_START + X_RAMP_END) / 2
    ramp_len = x_mid - THROAT_HALF_WIDTH - X_RAMP_START
    
    # Distance from throat edge, clipped to [0, ramp_len]
    dist = np.clip(np.abs(xx - x_mid) - THROAT_HALF_WIDTH, 0, ramp_len)
    xi = dist / ramp_len
    
    return np.where(
        (xx >= X_RAMP_START) & (xx <= X_RAMP_END),
        (H_INLET + H_THROAT)/2 - (H_INLET - H_THROAT)/2 * np.cos(np.pi * xi),
        H_INLET
    )

# Initialize problem
problem = Problem.from_yaml("../../tests/configs/bernoulli_venturi.yaml")

# Set Venturi topography and plot
h_venturi = venturi_topography(problem.topo.xx)
problem.topo.set_mapped_height(h_venturi)

#problem.plot_topo()

Now run the problem:

In [ ]:
problem.run()

Compare with Bernoulli predictions:

In [ ]:
# Extract centerline data
rho_b = problem.q[0][1:-1, 1:-1]
jx_b = problem.q[1][1:-1, 1:-1]
h_b = problem.topo.h[1:-1, 1:-1]
j_center = rho_b.shape[1] // 2

dx_b = problem.grid['dx']
x_b = np.arange(rho_b.shape[0]) * dx_b + dx_b / 2
vx_b = jx_b[:, j_center] / rho_b[:, j_center]
p_b = np.asarray(eos_pressure(rho_b[:, j_center], problem.prop))

# Theory: velocity from mass conservation, pressure from Bernoulli
v_theory = V_INLET * H_INLET / h_b[:, j_center]
p_ref = p_b[-1]
v_ref = V_INLET * H_INLET / h_b[-1, j_center]
p_theory = p_ref + 0.5 * RHO0 * (v_ref**2 - v_theory**2)

# Compute errors
inlet = x_b < X_RAMP_START
throat = (x_b >= 0.045) & (x_b <= 0.055)

v_in, v_throat = vx_b[inlet].mean(), vx_b[throat].mean()
p_in, p_throat = p_b[inlet].mean(), p_b[throat].mean()

v_throat_theory = V_INLET * H_INLET / H_THROAT
dp_theory = 0.5 * RHO0 * (v_throat_theory**2 - V_INLET**2)
dp_sim = p_in - p_throat

print(f"Velocity at throat: {v_throat:.2f} m/s (theory: {v_throat_theory:.2f} m/s)")
print(f"Velocity ratio: {v_throat/v_in:.3f} (theory: 2.0)")
print(f"Pressure drop: {dp_sim:.1f} Pa (theory: {dp_theory:.1f} Pa)")
print(f"Pressure drop error: {abs(dp_sim - dp_theory)/dp_theory*100:.2f}%")

In [ ]:
# Plot comparison
x_mm = x_b * 1000  # Convert to mm
fig, axes = plt.subplots(2, 2, figsize=(7, 5), facecolor='white', constrained_layout=True)

def vlines(ax):
    ax.axvline(X_RAMP_START*1000, color='gray', ls='--', alpha=0.5)
    ax.axvline(X_RAMP_END*1000, color='gray', ls='--', alpha=0.5)

# 1. Gap height
axes[0,0].plot(x_mm, h_b[:, j_center]*1000, 'b-', lw=2)
vlines(axes[0,0])
axes[0,0].set(xlabel='x [mm]', ylabel='h [mm]', title='Gap Height')

# 2. Velocity
axes[0,1].plot(x_mm, vx_b, 'b-', lw=2, label='Simulation')
axes[0,1].plot(x_mm, v_theory, 'r--', lw=1.5, label='Theory')
vlines(axes[0,1])
axes[0,1].set(xlabel='x [mm]', ylabel='v [m/s]', title='Velocity')
axes[0,1].legend()

# 3. Pressure
axes[1,0].plot(x_mm, p_b/1000, 'b-', lw=2, label='Simulation')
axes[1,0].plot(x_mm, p_theory/1000, 'r--', lw=1.5, label='Theory')
vlines(axes[1,0])
axes[1,0].set(xlabel='x [mm]', ylabel='p [kPa]', title='Pressure')
axes[1,0].legend()

# 4. Momentum flux
axes[1,1].plot(x_mm, jx_b[:, j_center], 'b-', lw=2)
vlines(axes[1,1])
axes[1,1].set(xlabel='x [mm]', ylabel='$j_x$ [kg/(m$^2$s)]', title='Momentum Flux')

fig.suptitle('Bernoulli Venturi Validation', fontweight='bold')
plt.show()

## Diffusion

Dynamic simulation of 2D heat diffusion with comparison to the analytic solution.

Setup:
- Dirichlet temperature boundary conditions in x: T=0 at both ends
- Periodic boundary conditions in y
- Initial temperature: sine profile $T(x,0) = T_{max} * sin(π*x_{norm})$ constant over y
- No wall movement/fluid flow $(U=0, V=0)$, pure diffusion

Since the initial condition and boundary conditions are uniform in y, and y has periodic BCs, the solution should match the 1D analytic solution:

$$T(x,t) = T_{max} \cdot e^{-\lambda^2 t} \cdot \sin\left(\pi \cdot x_{norm}\right)$$

where $\lambda = \sqrt{\alpha} \cdot \frac{\pi}{L_{eff}}$ and $\alpha = \frac{k}{c_v \rho}$.

$\rightarrow$ **[YAML File](../../tests/configs/temp_diffusion_2d.yaml)**

In [ ]:
# 2D Temperature diffusion
import numpy as np
from GaPFlow.problem import Problem

# Store temperature history for comparison
T_history = {'time': [], 'T': []}

problem = Problem.from_yaml("../../tests/configs/temp_diffusion_2d.yaml")

def track_temperature():
    dt = problem.numerics['dt']
    t = problem.step * dt
    T_history['time'].append(t)
    # Extract T along x at mid-y (should be constant in y)
    T_history['T'].append(problem.energy.temperature[:, problem.grid['Ny']//2 + 1].copy())

problem.add_callback(track_temperature)
problem.run()

Animation of the comparison to the analytic solution:

In [ ]:
# Compare with analytic solution
import sys; sys.path.insert(0, '.')
from utils.plotting import animate_comparison  # type: ignore
from GaPFlow.utils import heat_equation_1d

# Get grid parameters
L = problem.grid['Lx']
Nx = problem.grid['Nx']
dx = problem.grid['dx']
cv = problem.energy_spec['cv']
k = problem.energy_spec['k']
rho = problem.prop['rho0']
T_max = problem.energy_spec['T0'][2]

# Cell centers including ghost cells: -dx/2, dx/2, 3dx/2, ..., Lx+dx/2
x_vec = np.arange(Nx + 2) * dx - dx / 2
t_vec = np.array(T_history['time'])

# For half_sine_ghost: T = sin(pi * (x + dx/2) / (Lx + dx))
# Zeros at ghost cell centers: x = -dx/2 and x = Lx + dx/2
# This matches where FEM 2D enforces Dirichlet BC
L_eff = L + dx
x_shifted = x_vec + dx / 2  # shift so x_shifted=0 at x=-dx/2

T_analytic = heat_equation_1d(L_eff, cv, k, rho, T_max, x_shifted, t_vec)
T_numeric = np.array(T_history['T'])

animate_comparison(x_vec, t_vec, T_numeric, T_analytic, L)